In [1]:
import math
import numpy as np
import pandas as pd
import os
import glob

from pykrx import stock
from pykrx import bond

import time
from time import sleep
from datetime import datetime
from datetime import timedelta

from pyarrow import csv
import pyarrow as pa
import pyarrow.parquet as pq

from ta.trend import MACD
from ta.momentum import StochasticOscillator

import psycopg2 as pg2
from sqlalchemy import create_engine

from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go

from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import storage


import warnings
warnings.filterwarnings('ignore')

# 경로 변경
os.chdir('/home/shjj08choi4/finance_mlops')

In [2]:
file_list = ['./data_crawler/cleaning/kor_stock_ohlcv/df_raw_anal_total_2_KOSPI_20240107.parquet',
             './data_crawler/cleaning/kor_stock_ohlcv/df_raw_total_2_KOSPI_20240107.parquet']

In [3]:
t = pq.read_table(file_list[1])
df = t.to_pandas()

In [4]:

def macd_vis(df_raw, technical_indicator_nm):
    # fig = go.Figure()
    fig = make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.01, row_heights=[0.5, 0.1, 0.2, 0.2])

    # 캔들스틱차트
    fig.add_trace(go.Candlestick(
        x=df_raw['date'],
        open=df_raw['open'],
        high=df_raw['high'],
        low=df_raw['low'],
        close=df_raw['close'],
        increasing_line_color= 'red', decreasing_line_color= 'blue', 
        name = ''), row = 1, col = 1)

    # MA 5 
    fig.add_trace(go.Scatter(x=df_raw['date'],y=df_raw['MA5'],
                             opacity=0.7,
                             line=dict(color='blue', width=2),
                             name='MA 5') , row = 1, col = 1)

    # MA 20
    fig.add_trace(go.Scatter(x=df_raw['date'],y=df_raw['MA20'],
                             opacity=0.7,
                             line=dict(color='orange', width=2),
                             name='MA 20'), row = 1, col = 1)

    # MA 60
    fig.add_trace(go.Scatter(x=df_raw['date'],y=df_raw['MA60'],
                             opacity=0.7,
                             line=dict(color='purple', width=2),
                             name='MA 60'), row = 1, col = 1)

    # MA 120
    fig.add_trace(go.Scatter(x=df_raw['date'],y=df_raw['MA120'],
                             opacity=0.7,
                             line=dict(color='green', width=2),
                             name='MA 120'), row = 1, col = 1)

    fig.add_trace(go.Scatter(
        x=pd.concat([df_raw['date'], df_raw['date'][::-1]]),
        y=pd.concat([df_raw['upper'], df_raw['lower'][::-1]]),
        fill='toself',
        fillcolor='rgba(255,255,0,0.1)',
        line=dict(color='rgba(255,255,255,0.2)', width=2),
        name='Bollinger Band',
        showlegend=False
    ), row = 1, col = 1)

#     for type_nm in type_list:
#         index_list = df_raw_anal_date.index[df_raw_anal_date[technical_indicator_nm].str.contains(type_nm)]
#         for index_nm in index_list:
#             cross_date = df_raw['date'][index_nm]
#             cross_value = df_raw['close'][index_nm]

#             fig.add_annotation(x=cross_date, 
#                                y=cross_value,
#                                text=f'{type_nm} <br> {technical_indicator_nm}',
#                                showarrow=True,
#                                arrowhead=1,
#                                row = 1, col = 1)                

    # Row 2 
    # volume
    colors = ['blue' if row['open'] - row['close'] >= 0 
              else 'red' for index, row in df_raw.iterrows()]

    fig.add_trace(go.Bar(x=df_raw['date'], 
                         y=df_raw['volume'],
                         marker_color=colors,
                         name = 'Volume',
                         showlegend=False
                        ), row=2, col=1)

    # MACD
    colors = ['red' if val >= 0 
              else 'blue' for val in df_raw['MACD_DIFF']]
    fig.add_trace(go.Bar(x=df_raw['date'], 
                         y=df_raw['MACD_DIFF'],
                         marker_color=colors,
                         name =  'MACD DIFF'
                        ), row=3, col=1)

    fig.add_trace(go.Scatter(x=df_raw['date'],
                             y=df_raw['MACD'],
                             line=dict(color='green', width=2),
                             name = 'MACD'
                            ), row=3, col=1)
    fig.add_trace(go.Scatter(x=df_raw['date'],
                             y=df_raw['MACD_Signal'],
                             line=dict(color='orange', width=1),
                             name = 'MACD Signal'
                            ), row=3, col=1)

    # Row 5
    # RSI
    fig.add_trace(go.Scatter(x=df_raw['date'],
                             y=df_raw['RSI'],
                             line=dict(color='red', width=1),
                             name = 'RSI'
                            ), row=4, col=1)
    
    # 수평 사각 영역 추가하기
    
    fig.add_hrect(y0=70, y1=100, line_width=0, fillcolor="white", opacity=0.1,
                  annotation_text="과매수구간", 
                  annotation_position="top right",
                  annotation_font_size=10,
                  annotation_font_color="red",
                  annotation_font_family="Times New Roman", row=4, col=1)
    
    fig.add_hrect(y0=0, y1=30, line_width=0, fillcolor="white", opacity=0.1,
                  annotation_text="과매도 구간", 
                  annotation_position="top right",
                  annotation_font_size=10,
                  annotation_font_color="blue",
                  annotation_font_family="Times New Roman", row=4, col=1)    
    
    
    # Rayout
    fig.update_layout(
        title = '삼성전자 주가',
        title_font_family="맑은고딕",
        title_font_size = 18,
        hoverlabel=dict(
            bgcolor='black',
            font_size=15,
        ),
        hovermode="x unified",
        template='plotly_dark',
        xaxis_tickangle=90,
        yaxis_tickformat = ',',
        legend = dict(orientation = 'h', xanchor = "center", x = 0.5, y= 1.2),
        barmode='group',
        margin=go.layout.Margin(
            l=10, #left margin
            r=10, #right margin
            b=10, #bottom margin
            t=100  #top margin
        ),
        height=600, width=900, 
    #     showlegend=False, 
        xaxis_rangeslider_visible=False
    )

    # update y-axis label
    fig.update_yaxes(title_text="Price", row=1, col=1)
    fig.update_yaxes(title_text="Volume", row=2, col=1)
    fig.update_yaxes(title_text="MACD", row=3, col=1)
    fig.update_yaxes(title_text="RSI", row=4, col=1)

    fig.update_xaxes(rangebreaks=[dict(bounds=["sat", "mon"])])
    return fig


In [13]:
kor_stock_ohlcv_095570_total = df[df['ticker'] == '005930'].sort_values(by = 'date').reset_index(drop = True)

In [21]:
kor_stock_ohlcv_095570_total['date'] = kor_stock_ohlcv_095570_total['date'].dt.strftime('%Y-%m-%d')

In [30]:
kor_stock_ohlcv_095570_total[kor_stock_ohlcv_095570_total['date'] < '2023-10-25']

,date,open,high,low,close,volume,price_change_percentage,ticker,corp_name,market,...,MACD_Signal,변화량,상승폭,하락폭,AU,AD,RSI,MA5-20,MA20-60,MA60-120
0,2023-01-02,55500,56100,55200,55500.0,10031448,0.361664,005930,삼성전자,KOSPI,...,-51.423271,300.0,300.0,0.0,2524.744891,2792.099375,47.485779,-7343.5,-1257.333333,-317.083333
1,2023-01-03,55400,56000,54500,55400.0,13547030,-0.180180,005930,삼성전자,KOSPI,...,56.758129,200.0,200.0,0.0,2299.415740,2427.913626,48.640904,-6977.5,1627.500000,-2060.583333
2,2023-01-04,55700,58000,55600,57800.0,20188071,4.332130,005930,삼성전자,KOSPI,...,-1145.081517,1200.0,1200.0,0.0,2258.424768,2399.666463,48.483910,-10405.0,4880.500000,-1367.083333
3,2023-01-05,58200,58800,57600,58200.0,15682826,0.692042,005930,삼성전자,KOSPI,...,-59.694388,12900.0,12900.0,0.0,3210.535134,3358.790765,48.871607,-15852.5,166.000000,-226.500000
4,2023-01-06,58300,59400,57900,59000.0,17334989,1.374570,005930,삼성전자,KOSPI,...,-2176.220957,7100.0,7100.0,0.0,2610.883430,2665.237807,49.484902,-10303.5,3284.166667,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,2023-10-18,68900,70500,68800,70500.0,16493184,1.585014,005930,삼성전자,KOSPI,...,1397.333313,800.0,800.0,0.0,1934.969236,1215.608200,61.416336,10102.5,-26.166667,-531.083333
195,2023-10-19,69700,70000,69400,69500.0,13985012,-1.418440,005930,삼성전자,KOSPI,...,-219.623657,11300.0,11300.0,0.0,2260.235017,1417.527778,61.456792,987.0,965.333333,221.583333
196,2023-10-20,68900,69200,68100,68800.0,15204495,-1.007194,005930,삼성전자,KOSPI,...,-1852.822937,21150.0,21150.0,0.0,4195.060418,3349.478045,55.603937,-2785.0,94.000000,-28.666667
197,2023-10-23,68700,69100,68200,68400.0,11625959,-0.581395,005930,삼성전자,KOSPI,...,-991.834899,0.0,0.0,0.0,2859.158769,2320.595489,55.198734,7407.5,-6400.833333,55.250000


In [26]:
macd_vis(kor_stock_ohlcv_095570_total[kor_stock_ohlcv_095570_total['date'] > '2023-08-01'],'a')

In [18]:
ticker_nm = '005930' # 삼성전자 ticker
start_date  = '20190101'
today_date1 = '20231024'

df_raw = stock.get_market_ohlcv(start_date, today_date1, ticker_nm)
df_raw = df_raw.reset_index()
df_raw['ticker'] = ticker_nm

df_raw.columns = ['date', 'open', 'high', 'low', 'close', 'volume','price_change_percentage', 'ticker']

In [ ]:
kor_stock_ohlcv_095570_total

In [31]:
import math
import numpy as np
import pandas as pd
import os
import glob

from pykrx import stock
from pykrx import bond

import time
from time import sleep
from datetime import datetime
from datetime import timedelta

from pyarrow import csv
import pyarrow as pa
import pyarrow.parquet as pq

from ta.trend import MACD
from ta.momentum import StochasticOscillator

import psycopg2 as pg2
from sqlalchemy import create_engine

from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go

from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import storage


import warnings
warnings.filterwarnings('ignore')

# 경로 변경
os.chdir('/home/shjj08choi4/finance_mlops')


# 서비스 계정 키 JSON 파일 경로
key_path = glob.glob("key_value/*.json")[0]

# Credentials 객체 생성
credentials = service_account.Credentials.from_service_account_file(key_path)

# 빅쿼리 정보
project_id = 'owenchoi-404302'
dataset_id = 'finance_mlops'


# BigQuery 클라이언트 객체 생성
client = bigquery.Client(credentials = credentials, 
                         project = credentials.project_id)


# GCP 클라이언트 객체 생성
storage_client = storage.Client(credentials = credentials,
                         project = credentials.project_id)
bucket_name = 'finance-mlops-proj'    # 서비스 계정 생성한 bucket 이름 입력


now = datetime.now()
# now = now + timedelta(days=-2)
today_date1 = now.strftime('%Y%m%d')
today_date2 = now.strftime('%Y-%m-%d')
today_date_time_csv = now.strftime("%Y%m%d_%H%M")

today_date1 = '2024010'
today_date2 = '2023-01-07'

today_date1 = 'reset'

sql = f"""
select 
  `date`,
  open, 
  high, 
  low, 
  close, 
  volume,
  price_change_percentage,
  `{project_id}.{dataset_id}.kor_stock_ohlcv`.ticker,
  corp_name, 
  market 
from `{project_id}.{dataset_id}.kor_stock_ohlcv`
left join  `{project_id}.{dataset_id}.kor_ticker_list`
on `{project_id}.{dataset_id}.kor_stock_ohlcv`.ticker = `{project_id}.{dataset_id}.kor_ticker_list`.ticker
order by date asc
"""

In [32]:
sql

'\nselect \n  `date`,\n  open, \n  high, \n  low, \n  close, \n  volume,\n  price_change_percentage,\n  `owenchoi-404302.finance_mlops.kor_stock_ohlcv`.ticker,\n  corp_name, \n  market \nfrom `owenchoi-404302.finance_mlops.kor_stock_ohlcv`\nleft join  `owenchoi-404302.finance_mlops.kor_ticker_list`\non `owenchoi-404302.finance_mlops.kor_stock_ohlcv`.ticker = `owenchoi-404302.finance_mlops.kor_ticker_list`.ticker\norder by date asc\n'